In [1]:
import pickle as pkl
import cv2
import ipywidgets as widgets
from IPython.display import display
import os
import re
from collections import defaultdict
from pprint import pprint
import torch
import numpy as np
import networkx as nx

In [2]:
video_id = "00T1E.mp4"

In [3]:
pred = pkl.load(open(f'output/sgcls/{video_id}/pred.pkl', 'rb'))
semi_con = pkl.load(open(f'output/sgcls/{video_id}/semi_constraint.pkl', 'rb'))

In [4]:
semi_con.keys()

dict_keys(['00T1E.mp4/000010.png', '00T1E.mp4/000013.png', '00T1E.mp4/000022.png', '00T1E.mp4/000028.png', '00T1E.mp4/000038.png', '00T1E.mp4/000041.png', '00T1E.mp4/000062.png', '00T1E.mp4/000063.png', '00T1E.mp4/000065.png', '00T1E.mp4/000077.png', '00T1E.mp4/000087.png', '00T1E.mp4/000096.png', '00T1E.mp4/000111.png', '00T1E.mp4/000125.png', '00T1E.mp4/000137.png', '00T1E.mp4/000162.png', '00T1E.mp4/000163.png', '00T1E.mp4/000187.png', '00T1E.mp4/000190.png', '00T1E.mp4/000201.png', '00T1E.mp4/000207.png', '00T1E.mp4/000224.png', '00T1E.mp4/000239.png', '00T1E.mp4/000240.png', '00T1E.mp4/000257.png', '00T1E.mp4/000273.png', '00T1E.mp4/000285.png', '00T1E.mp4/000297.png', '00T1E.mp4/000310.png'])

In [5]:
pred.keys()

dict_keys(['boxes', 'labels', 'scores', 'distribution', 'pred_labels', 'im_idx', 'pair_idx', 'human_idx', 'features', 'attention_gt', 'spatial_gt', 'contacting_gt', 'fmaps', 'im_info', 'indices', 'object_features', 'object_mem_features', 'pred_scores', 'union_feat', 'union_box', 'spatial_masks', 'rel_features', 'rel_mem_features', 'attention_distribution', 'spatial_distribution', 'contacting_distribution'])

In [6]:
for k in pred.keys():
    print(f"{k} -- {len(pred[k]) if type(pred[k]) == list else pred[k].shape}")


boxes -- torch.Size([80, 5])
labels -- torch.Size([80])
scores -- torch.Size([80])
distribution -- torch.Size([80, 36])
pred_labels -- torch.Size([80])
im_idx -- torch.Size([51])
pair_idx -- torch.Size([51, 2])
human_idx -- torch.Size([29, 1])
features -- torch.Size([80, 2048])
attention_gt -- 51
spatial_gt -- 51
contacting_gt -- 51
fmaps -- torch.Size([29, 1024, 67, 38])
im_info -- torch.Size([])
indices -- 6
object_features -- torch.Size([80, 2376])
object_mem_features -- torch.Size([80, 2376])
pred_scores -- torch.Size([80])
union_feat -- torch.Size([51, 1024, 7, 7])
union_box -- torch.Size([51, 5])
spatial_masks -- torch.Size([51, 2, 27, 27])
rel_features -- torch.Size([51, 1936])
rel_mem_features -- torch.Size([51, 1936])
attention_distribution -- torch.Size([51, 3])
spatial_distribution -- torch.Size([51, 6])
contacting_distribution -- torch.Size([51, 17])


In [7]:
boxes = pred["boxes"]
labels = pred["labels"]
scores = pred["scores"]
distribution = pred["distribution"]
pred_labels = pred["pred_labels"]
im_idx = pred["im_idx"]
pair_idx = pred["pair_idx"]
human_idx = pred["human_idx"]
features = pred["features"]
attention_gt = pred["attention_gt"]
spatial_gt = pred["spatial_gt"]
contacting_gt = pred["contacting_gt"]
fmaps = pred["fmaps"]
im_info = pred["im_info"]
indices = pred["indices"]
object_features = pred["object_features"]
object_mem_features = pred["object_mem_features"]
pred_scores = pred["pred_scores"]
union_feat = pred["union_feat"]
union_box = pred["union_box"]
spatial_masks = pred["spatial_masks"]
rel_features = pred["rel_features"]
rel_mem_features = pred["rel_mem_features"]
attention_distribution = pred["attention_distribution"]
spatial_distribution = pred["spatial_distribution"]
contacting_distribution = pred["contacting_distribution"]


In [8]:
# Load txt file
object_classes = []
with open('/home/pali/datasets/charades/dataset/ag/annotations/object_classes.txt', 'r') as file:
    for line in file:
        object_classes.append(line.strip()) 
        
relationship_classes = []
with open('/home/pali/datasets/charades/dataset/ag/annotations/relationship_classes.txt', 'r') as file:
    for line in file:
        relationship_classes.append(line.strip())

In [9]:
# Path to the directory containing the frames
frames_dir = f'/home/pali/datasets/charades/dataset/ag/frames/{video_id}/'

# Load and sort the frames
frame_files = os.listdir(frames_dir)
frame_files = sorted(frame_files, key=lambda x: int(re.search(r'\d+', x).group()))

frames = []
for frame_file in frame_files:
    frame_path = os.path.join(frames_dir, frame_file)
    frame = cv2.imread(frame_path)
    if frame is not None:
        frames.append(frame)

print(f"Total frames: {len(frames)}")


Total frames: 36


In [10]:
def interactive_plot(data, frames, threshold=0.7):
    # Function to display the frame with bounding boxes
    def display_frame(frame_index):
        frame = frames[frame_index].copy()
        
        # # Draw bounding boxes for the current frame
        # for bbox in frame_bboxes[frame_index]:
        #     x1, y1, x2, y2, label, score = bbox
        #     cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
            
        #     text = f'{label} {score:.2f}'

        #     # Calculate the size of the label to create a background for readability
        #     (label_width, label_height), baseline = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2)
        #     cv2.rectangle(frame, (int(x1), int(y1) - label_height - baseline), (int(x1) + label_width, int(y1)), (0, 255, 0), cv2.FILLED)
        #     cv2.putText(frame, text, (int(x1), int(y1) - baseline), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2)
        
        for sgg in data[frame_index]:
            
            
            # subject = sgg["subject"]
            # object = sgg["object"]
            x1, y1, x2, y2, = sgg["box"]
            # score = sgg["score"]
            pred_label = sgg["pred_label"]
            pred_score = sgg["pred_score"]
            # attn = sgg["attn"]
            # spat = sgg["spat"]
            # cont = sgg["cont"]

            # for object in sgg:


            # if pred_score > threshold:
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
                # pprint.pprint(sgg)
                # print(f'<{subject}, {object}>')
                # max_attn = torch.argmax(attn)
                # max_spat = torch.argmax(spat)
                # max_cont = torch.argmax(cont)
                # print(f'Attn: {relationship_classes[max_attn]} - score: {attn[max_attn]}')
                # print(f'Spat: {relationship_classes[max_spat]} - score: {spat[max_spat]}')
                # print(f'Cont: {relationship_classes[max_cont]} - score: {cont[max_cont]}')
            pprint({
                "pred_label": object_classes[pred_label-1],
                "pred_score": pred_score
            })
            print('*'*60)
            # print(f'<{subject}, {object}>')

        # Display the frame with bounding boxes
        frame_widget = widgets.Image(value=cv2.imencode('.jpg', frame)[1].tobytes())
        display(frame_widget)

    # Create the interactive widgets
    frame_index_slider = widgets.IntSlider(min=0, max=len(data)-1, value=0)

    # Update the frame when the slider value changes
    return widgets.interactive(display_frame, frame_index=frame_index_slider)

In [11]:
data = defaultdict(list)

In [12]:
data = defaultdict(list)
for i, box in enumerate(boxes):
    im_i = int(box[0])
    bbox = box[1:]
    score = scores[i]
    pred_label = pred_labels[i]
    pred_score = pred_scores[i]

    data[im_i].append({
        'box': bbox,
        'score': score,
        'pred_label': pred_label,
        'pred_score': pred_score,
    })


In [13]:
interactive_plot(data, frames, threshold=0.967)

interactive(children=(IntSlider(value=0, description='frame_index', max=28), Output()), _dom_classes=('widget-…

In [14]:
# First, prepare a dictionary for quick access to box information by its index
box_info_by_idx = {}
for i, box in enumerate(boxes):
    im_i = int(box[0])
    bbox = box[1:]
    score = scores[i].item()
    pred_label = pred_labels[i].item()  # Assuming these are indices into some label list
    box_info_by_idx[i] = {
        'im_idx': im_i,
        'box': bbox.numpy().tolist(),  # Convert tensors to list for JSON-serializability if needed
        'score': score,
        'pred_label': pred_label
    }

In [15]:
# Iterate over pair_idx to form triplets
triplets = defaultdict(list)
for i, pair in enumerate(pair_idx):
    idx1, idx2 = pair.tolist()  # Convert tensor to list
    im_i = box_info_by_idx[idx1]['im_idx']  # Assuming both boxes are from the same image

    # Retrieve object and relation information
    object1 = box_info_by_idx[idx1]
    object2 = box_info_by_idx[idx2]
    relation = attention_gt[i]  # Or spatial_gt[i], contacting_gt[i], as needed

    # Append the triplet to the list for the corresponding image index
    triplets[im_i].append({
        'object1': object1,
        'relation': relation,
        'object2': object2
    })


In [16]:
triplets

defaultdict(list,
            {0: [{'object1': {'im_idx': 0,
                'box': [95.98867797851562,
                 163.20693969726562,
                 181.31173706054688,
                 231.32737731933594],
                'score': 0.9201374650001526,
                'pred_label': 1},
               'relation': [0],
               'object2': {'im_idx': 0,
                'box': [146.0, 187.5, 209.0, 235.5],
                'score': 0.9349483251571655,
                'pred_label': 30}},
              {'object1': {'im_idx': 0,
                'box': [95.98867797851562,
                 163.20693969726562,
                 181.31173706054688,
                 231.32737731933594],
                'score': 0.9201374650001526,
                'pred_label': 1},
               'relation': [1],
               'object2': {'im_idx': 0,
                'box': [0.0, 109.25, 269.9644775390625, 342.375],
                'score': 0.9996798038482666,
                'pred_label': 3}}],
      

In [17]:
interactive_plot(triplets, frames, threshold=0.967)

interactive(children=(IntSlider(value=0, description='frame_index', max=28), Output()), _dom_classes=('widget-…

In [18]:
triplets[0]

[{'object1': {'im_idx': 0,
   'box': [95.98867797851562,
    163.20693969726562,
    181.31173706054688,
    231.32737731933594],
   'score': 0.9201374650001526,
   'pred_label': 1},
  'relation': [0],
  'object2': {'im_idx': 0,
   'box': [146.0, 187.5, 209.0, 235.5],
   'score': 0.9349483251571655,
   'pred_label': 30}},
 {'object1': {'im_idx': 0,
   'box': [95.98867797851562,
    163.20693969726562,
    181.31173706054688,
    231.32737731933594],
   'score': 0.9201374650001526,
   'pred_label': 1},
  'relation': [1],
  'object2': {'im_idx': 0,
   'box': [0.0, 109.25, 269.9644775390625, 342.375],
   'score': 0.9996798038482666,
   'pred_label': 3}}]

In [19]:
for k, v in semi_con.items():
    pprint(v)

{'triplet_boxes': array([[ 95.98867798, 163.2069397 , 181.31173706, 231.32737732,
        146.        , 187.5       , 209.        , 235.5       ],
       [ 95.98867798, 163.2069397 , 181.31173706, 231.32737732,
          0.        , 109.25      , 269.96447754, 342.375     ],
       [146.        , 187.5       , 209.        , 235.5       ,
         95.98867798, 163.2069397 , 181.31173706, 231.32737732],
       [  0.        , 109.25      , 269.96447754, 342.375     ,
         95.98867798, 163.2069397 , 181.31173706, 231.32737732],
       [  0.        , 109.25      , 269.96447754, 342.375     ,
         95.98867798, 163.2069397 , 181.31173706, 231.32737732],
       [  0.        , 109.25      , 269.96447754, 342.375     ,
         95.98867798, 163.2069397 , 181.31173706, 231.32737732],
       [ 95.98867798, 163.2069397 , 181.31173706, 231.32737732,
          0.        , 109.25      , 269.96447754, 342.375     ]]),
 'triplet_labels': array([[ 1,  0, 30],
       [ 1,  1,  3],
       [30,  4, 

In [22]:
semi_con["00T1E.mp4/000013.png"]

{'triplet_scores': array([[0.999933  , 0.9999609 , 0.9999997 ],
        [0.999933  , 0.99990857, 1.        ],
        [0.9999609 , 0.999933  , 0.99724919],
        [0.9999609 , 0.999933  , 0.9048481 ],
        [0.99990857, 0.999933  , 0.99167377],
        [0.99990857, 0.999933  , 0.99997556],
        [0.99990857, 0.999933  , 0.99636543],
        [0.999933  , 0.9999609 , 0.94446832],
        [0.999933  , 0.9999609 , 0.93607271],
        [0.999933  , 0.99990857, 0.99999988]]),
 'triplet_labels': array([[ 1,  0, 30],
        [ 1,  1,  3],
        [30,  4,  1],
        [30,  5,  1],
        [ 3,  3,  1],
        [ 3,  6,  1],
        [ 3,  7,  1],
        [ 1, 21, 30],
        [ 1, 23, 30],
        [ 1, 16,  3]]),
 'triplet_boxes': array([[ 15.68480015, 163.501297  , 176.19741821, 242.45726013,
         145.        , 189.51010132, 209.09434509, 236.7339325 ],
        [ 15.68480015, 163.501297  , 176.19741821, 242.45726013,
           0.83333331, 109.16666412, 270.        , 343.33334351],
 

In [30]:
object_classes[6]

'broom'

In [21]:
for i, im_i in enumerate(im_idx):
    obj_1, obj_2 = pair_idx[i]

    graph = nx.DiGraph([
        (obj_1, obj_2, {'relation': attention_gt[i]})
    ])

In [41]:
graph